# New York City Taxi Trip Duration
<hr>

![](imgs/taxi.png)

# 1. Introdução <a id="intro"></a>

O dataset dessa aula foi extraído da competição do Kaggle [New York City Taxi Trip Duration](https://www.kaggle.com/c/nyc-taxi-trip-duration/overview). Nessa competição, o objetivo era prever a duração de uma viagem de táxi em Nova York, usando como dados as coordenadas geográficas do ponto de partida e de chegada e o momento da partida.

Mas peraí, por que prever a duração de uma viagem quando a gente já sabe o ponto de chegada?? Nem toda competição do Kaggle faz sentido na vida real, e foi feito até um disclaimer na descrição dos dados:

*Disclaimer: The decision was made to not remove dropoff coordinates from the dataset order to provide an expanded set of variables to use in Kernels.*

Mas há uma situação em que a competição faz muito sentido! Quando você pede um Uber, coloca logo o ponto de partida e o de chegada, e o aplicativo te dá uma previsão do tempo da viagem. É nesse cenário que vamos pensar!


**Importante**: O dataset que usaremos nessa aula não é exatamente o da competição. Foram adicionadas duas colunas com a região e o bairro de Nova York em que a corrida começou/terminou (informações obtidas [aqui](https://data.cityofnewyork.us/City-Government/Borough-Boundaries/tqmj-j8zm)). Além disso, para deixar a execução mais rápida, usaremos uma versão menor do dataset. Se quise ver como os dados foram preparados veja o notebook [prepare_dataset.ipynb](prepare_dataset.ipynb).

A métrica utilizada na competição é a **Root Mean Squared Logarithmic Error** (RMSLE), definida como

$$RMSLE = \sqrt{\frac1n \sum_{i=1}^n\big(\log(p_i+1)-\log(a_i+i)\big)^2}$$

onde $p_i$ é a predição para o exemplo $i$ e $a_i$ o ground-truth. Essa métrica motiva a transformação do target com a função $f(x) =\log(x+1)$.

# 2. Importando as bibliotecas <a id="import"></a>

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import holidays
from scipy.stats import probplot
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_log_error

# 3. Leitura dos dados <a id="read"></a>

Pra começar, leia os arquivos '../data/train_with_nta.csv' e '../data/test_with_nta.csv', e em seguida dê um .head(), .info() e .describe() no train


In [ ]:
train = pd.read_csv('../data/prepared_train.csv')
test = pd.read_csv('../data/prepared_test.csv')

In [ ]:
# head, describe, info
# TODO

Vemos que as colunas **pickup_datetime** e **dropoff_datetime** estão como object. Faça a conversão para **datetime** do pandas.

In [ ]:
# converter as colunas para datetime
# TODO

Se você já souber que alguma coluna é datetime, pode fazer a conversão na hora da leitura, dessa forma:

```python
train = pd.read_csv('../data/prepared_train.csv', parse_dates=['pickup_datetime', 'dropoff_datetime'])
```

Cheque se os datasets possuem algum **NaN**:

In [ ]:
# cheque a presença de missing values no treino e no teste

# 3. EDA

Tudo carregado na memória! Agora vamos começar a EDA. Você notou alguma coisa estranha no train.describe()? Plote a distribuição do target no train usando **sns.displot**

In [ ]:
# plotar distribuição do target no treino
# TODO

Plote um boxplot do target no treino, usando **sns.boxplot**

In [ ]:
# plotar boxplot do target no treino
# TODO

Qual o valor máximo da trip_duration no treino, em horas?

In [ ]:
# achar valor maximo de trip_duration
# TODO

Quantas viagens duraram mais que 4h?

In [ ]:
# contar número de viagens com mais de 4h
# TODO

Remova as viagens com duração anormal.

In [ ]:
# remover outliers no trip_duration
# TODO

Plote novamente a distribuição do target e o boxplot (coloque o eixo x em horas ao invés de segundos)

In [ ]:
# plotar novamente distribuição e boxplot do target no treino
# TODO

Em problemas de regressão, é comum aplicar a função log(x) ou log(x+1) no target. Plote a distribuição do log do target.

In [ ]:
# plotar a ditribuição do log do target

Essa distribuição parece gaussiana. Um dos jeitos de checar visualmente se seus dados seguem uma normal é fazer um [probability plot](https://en.wikipedia.org/wiki/Normal_probability_plot). Compare os plots feitos com variáveis geradas de uma distribuição normal e de uma exponencial:

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 5))
_ = probplot(np.random.normal(size=1000), plot=axs[0])
_ = probplot(np.random.exponential(size=1000), plot=axs[1])

Compare os plots feitos com o target e com o log do target:

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 5))
_ = probplot(train.trip_duration, plot=axs[0])
_ = probplot(np.log(train.trip_duration), plot=axs[1])

Há algum outlier no número de passageiros? Faça um describe e um histograma.

In [ ]:
# checar distribuição do número de passageiros

Como viram na aula de regressão linear, os táxis formam um mapa da cidade:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
train.plot(kind='scatter', x='pickup_longitude', y='pickup_latitude', s=0.2, alpha=0.1, color='salmon', ax=ax)
ax.set_facecolor('black')

A figura é melhor visualizada se limitarmos os eixos x e y:

In [ ]:
long_interval = (-74.04, -73.75)
lat_interval = (40.63, 40.88)

fig, ax = plt.subplots(figsize=(10, 10))
train.plot(kind='scatter', x='pickup_longitude', y='pickup_latitude', s=0.2, alpha=0.1, color='salmon', ax=ax)
ax.set_facecolor('black')

Uma imagem parecida também pode ser gerada coma biblioteca [Datashader](https://datashader.org/). Ela gera a imagem bem mais rápido do que o .scatter e dá muitas opções pra manipulação, não faz parte da aula, mas aqui vai o código como exemplo e a figura que ele geraria:

```python
from colorcet import fire
import datashader as ds 
import datashader.transfer_functions as dtf
from datashader.utils import lnglat_to_meters

data = pd.read_csv('../data/train.csv')
x, y = lnglat_to_meters(data.pickup_longitude, data.pickup_latitude)
mercator_df = pd.DataFrame({'x': x, 'y': y})

west, south, east, north = 40.635, -74.03, 40.86, -73.77
bottom_left = lnglat_to_meters(south, west)
top_right = lnglat_to_meters(north, east)
x_range = (bottom_left[0], top_right[0])
y_range = (bottom_left[1], top_right[1])

canvas = ds.Canvas(plot_width=600, plot_height=600, x_range=x_range, y_range=y_range)
agg = canvas.points(mercator_df, 'x', 'y')
dtf.set_background(dtf.shade(agg, cmap=fire), 'black')
```

Output:

![](imgs/datashader.png)

A coluna de datetime bruta não pode ser usada diretamente como feature. Por isso, vamos separá-la em hora, mês, dia da semana e ano. Crie as colunas abaixo usando as [propriedades .dt](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) do pandas

In [ ]:
train['hour'] = #
train['day'] = #
train['weekday'] = #
train['month'] = #

test['hour'] = #
test['day'] = #
test['weekday'] = #
test['month'] = #

Com esses features iniciais já podemos tirar informações interessantes sobre o trânsito de Nova York. Use a função **sns.countplot**, vista na aula de regressão linear, pra plotar a distribuição do número de corridas pelas horas do dia.

In [ ]:
# countplot na hora
# TODO

O que está por baixo dos panos do sns.countplot é um groupby. Tente reproduzir o gráfico acima usando grouby e a função de agregação **size**.

In [ ]:
# groupby na hora
# TODO

Use a função **sns.catplot** para plotar a média da duração das viagens a cada hora do dia (um gráfico parecido foi feito na aula de regressão linear)

In [ ]:
# catplot trip_duration vs hour
# TODO

Tente reproduzir o gráfico acima usando groupby (não precisa das barras de erro nem das cores).

In [ ]:
# reproduzir catplot trip_duration vs hour
# TODO

Qual região tem mais viagens iniciadas? Responda essa pergunta com um gráfico, usando countplot ou groupby em **pickup_neighborhood**.

In [ ]:
# countplot no pickup_neighborhood
# TODO

Faço o mesmo para viagens finalizadas.

In [ ]:
# countplot no dropoff_neighborhood
# TODO

Como visualizar a série temporal do númerod e viagens a cada dia? Uma maneira possível é utilizar a propriedade **dayoftheyear** e realizar um groupby (note a key passada para o groupby não precisa ser uma coluna do dataframe, basta ter o mesmo tamanho):

In [ ]:
train.groupby(train.pickup_datetime.dt.dayofyear).size().plot()

Mas e se quisermos plotar o número de viagens a cada semana, a cada 15 dias, a cada 17 minutos? Para isso usamos o [**pd.Grouper**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Grouper.html), que nos auxilia a fazer groupby sobre datas, especificando-se a frequência. Por exemplo, o gráfico acima pode ser feito assim:

In [ ]:
train.groupby(pd.Grouper(key='pickup_datetime', freq='d')).size().plot()

Se quisermos plotar a contagem a cada 15 dias ou a cada semana, basta mudar o paramêtro de frequência para '15d' ou 'w':

In [ ]:
train.groupby(pd.Grouper(key='pickup_datetime', freq='15d')).size().plot()

In [ ]:
train.groupby(pd.Grouper(key='pickup_datetime', freq='w')).size().plot()

Podemos, é claro, usar outras funções de agregação:

In [ ]:
train.groupby(pd.Grouper(key='pickup_datetime', freq='w')).mean().trip_duration.plot()

E agrupar em mais de uma coluna (não se preocupe muito com o comando unstack):

In [ ]:
train.groupby([pd.Grouper(key='pickup_datetime', freq='d'), 'vendor_id']).size().unstack().plot()

Vendo esses gráficos, parece ter um dia bem anormal no final de janeiro. Qual foi esse dia?

In [ ]:
# Encontrar dia anormal
# TODO

Pesquise no google o que aconteceu em Nova York nesse dia.

Essa conclusão nos motiva a procurar features baseados nas condições climáticas nos dias de 2016. Uma possibilidade é procurar alguma API que forneça essas informações, mas felizmente participantes da competição no Kagge já fizeram esse trabalho! Vamos usar [esse dataset](https://www.kaggle.com/cabaki/knycmetars2016), que da deve estar na sua pasta data

In [ ]:
weather = pd.read_csv('../data/KNYC_Metars.csv', parse_dates=['Time'])
weather.head()

Faça um gráfico da média temperatura (coluna 'Temp.') por dia no ano de 2016. Use o pd.Grouper

In [ ]:
# Usar o pd.Grouper pra plotar a evolução da temperatura ao longo do ano de 2016
# TODO

Separe a coluna 'Time' em hora, dia, mês e ano:

In [ ]:
weather['year'] = #
weather['month'] = #
weather['day'] = #
weather['hour'] = #

Filtre o dataset removendo todas as colunas que não são de 2016:

In [ ]:
# Filtre o dataset para o ano de 2016
# TODO

Algumas manipulações finais:

In [ ]:
weather['snow'] = weather.Events.str.contains('Snow').astype(int) # converte para 1 todas as strings que contem "Snow"
weather = weather[['month','day','hour','Temp.','Precip','snow','Visibility']]

O merge com o dataset original pode ser feito assim:

In [ ]:
train = pd.merge(train, weather, on=['month', 'day', 'hour'], how='left')
test = pd.merge(test, weather, on=['month', 'day', 'hour'], how='left')

O dataset ficou assim:

In [ ]:
train.head()

In [ ]:
test.head()

Esse merge introduz alguns **NaNs**, pois provalmente há falta de dados para algumas horas.

In [ ]:
train.isnull().mean()

Uma estratégia seria preencher cada missing value com a média naquele dia ou semana, mas vamos adotar uma estratégia mais simples: back fill, que consiste em preencher um missing value com o próximo valor não faltante na coluna.

Use o método `fillna` com a estratégia 'bfill' nas colunas `Temp.`, `Precip` e `Visibility`.

In [ ]:
# preencher os missing values do treino e teste com bfill
# TODO

In [ ]:
# # Vamos preencher os nans da coluna neve com zeros
test.snow = test.snow.fillna(0)
train.snow = train.snow.fillna(0)

# 4. Feature Engineering

Vamos começar a criar features mais interessantes! Para começar, construa colunas com as variações de latitude de longitude.

In [ ]:
train['delta_lat'] =  # TODO
train['delta_long'] = # TODO

test['delta_lat'] =   # TODO
test['delta_long'] =  # TODO

**Feature grátis:** distância real em quilômetros. Como ninguém é obrigado a conhecer a [fórmula de Haversine](https://en.wikipedia.org/wiki/Haversine_formula), que dá a distância entre dois pontos numa circunferência, este feature já está pronto aqui! A implementação igual à [dessa biblioteca](https://github.com/mapado/haversine), só que feita pra aceita numpy arrays como entrada.

In [ ]:
def np_haversine(lat1, long1, lat2, long2):
    lat1, long1, lat2, long2 = map(np.radians, (lat1, long1, lat2, long2))
    AVG_EARTH_RADIUS = 6371  # radio da Terra em km
    delta_lat = lat2 - lat1
    delta_long = long2 - long1
    d = np.sin(delta_lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(delta_long * 0.5) ** 2
    return 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))

In [ ]:
train['haversine_dist']  = np_haversine(train['pickup_latitude'],
                                       train['pickup_longitude'],
                                       train['dropoff_latitude'],
                                       train['dropoff_longitude'])

test['haversine_dist']  = np_haversine(test['pickup_latitude'],
                                       test['pickup_longitude'],
                                       test['dropoff_latitude'],
                                       test['dropoff_longitude'])

Esse feature provavelmente é importante, mas será que a distância percorrida em linha reta tem uma relação tão direta com o tempo? Resposta óbvia pra quem mora em São Paulo: **não**. Use **sns.jointplot** e plote um gráfico de **haversine_dist** vs **trip_duration** (em horas).

In [ ]:
# jointplot do target vs haversine
# TODO

Como temos a distância em linha reta e o tempo da viagem, podemos criar uma coluna com a velocidade média do deslocamento. Essa coluna **não** poderá ser usada diretamente como feature. Por quê?

In [ ]:
train['average_speed'] = 3600 * train.haversine_dist / train.trip_duration

Apesar de não poder usar essa coluna diretamente como feature, vamos usá-la para entender melhor o dataset. Use groupby para plotar a média das velocidades ao longo das horas do dia.

In [ ]:
# plotar média das velocidades ao longo do dia
# TODO

Faça o mesmo para os dias da semana.

In [ ]:
# plotar média das velocidades ao longo da semana
# TODO

A informação da velocidade não está presente no teste, mas nada nos impede de utilizá-la para criar informações sobre cada região usando o conjunto de treino. Por exemplo, podemos calcular a velocidade média das viagens começando em cada região a cada hora e usar esse dado como feature! Isso pode ser feito com um groupby no código da área e na hora:

In [ ]:
train.groupby(['pickup_ntacode', 'hour']).mean().average_speed.reset_index()

In [ ]:
pickup_speed_nta_hour = train.groupby(['pickup_ntacode', 'hour']).mean().average_speed.reset_index()
pickup_speed_nta_hour = pickup_speed_nta_hour.rename(columns={'average_speed': 'pickup_speed_nta_hour'})
pickup_speed_nta_hour.head()

Tente fazer a mesma coisa com as regiões de dropoff:

In [ ]:
dropoff_speed_nta_hour = train.groupby(['dropoff_ntacode', 'hour']).agg({'average_speed': 'mean'}).reset_index()
dropoff_speed_nta_hour = dropoff_speed_nta_hour.rename(columns={'average_speed': 'dropoff_speed_nta_hour'})
dropoff_speed_nta_hour.head()

Agora basta juntar esses features com os datasets usando um pd.merge:

In [ ]:
train = pd.merge(train, pickup_speed_nta_hour, on=['pickup_ntacode', 'hour'], how='left')
test = pd.merge(test, pickup_speed_nta_hour, on=['pickup_ntacode', 'hour'], how='left')

In [ ]:
train = pd.merge(train, dropoff_speed_nta_hour, on=['dropoff_ntacode', 'hour'], how='left')
test = pd.merge(test, dropoff_speed_nta_hour, on=['dropoff_ntacode', 'hour'], how='left')

Se muitos táxis estão saindo de uma região A e indo para uma região B na mesma hora, pode ser um indicativo que o trânsito será mais lento. Usando o conjunto de treino, podemos construir essa tabela de "táxis na mesma direção" por hora. Tente construir essa tabela usando groupby e size(). Ela deve ficar mais ou menos assim:

| pickup_ntacode | dropoff_ntacode | hour | same_direction_by_hour |
|----------------|-----------------|------|------------------------|
| A              | B               | 0    | 12                     |
| C              | D               | 1    | 231                    |
| E              | F               | 2    | 67                     |


Use reset_index() depois do groupby para deixar o df nesse formato. Quando se usa a função de agregação size(), é criada uma coluna com o nome "0". Mude esse nome para algo mais apropriado.

In [ ]:
# Construir a tabela acima com groupby
# (pode mudar o nome se quiser)
same_direction = # TODO

Faça o merge dessa tabela com o treino e com o teste. 

In [ ]:
# backup do dataset antes do merge
train_copy = train.copy()
test_copy = test.copy()

In [ ]:
# merge da tabela same_direction com treino e teste (use how='left')
# TODO

Assegure-se que o dataset tem o mesmo número de linhas após o merge:

In [ ]:
assert len(train_copy) == len(train)
assert len(test_copy) == len(test)

In [ ]:
del train_copy, test_copy

Confira se adição desses features criou **NaNs** no treino ou no teste, o que pode ocorrer por exemplo caso exista no teste um taxi percursso de taxi que não existe no treino.

In [ ]:
# conferir se agora há NaNs no treino ou no teste
# TODO

Use alguma estratégia para eliminar esses NaNs.

In [ ]:
# eliminar NaNs, caso existam
# TODO

Sabemos que o trânsito é bem diferente em feriados. Vamos criar uma coluna binária `holiday` que diz se a data de pickup é ou não feriado em NY. Para isso, usaremos a biblioteca [holidays](https://github.com/dr-prodigy/python-holidays). Dê uma olhada no readme da repositório e tente criar essa coluna no treino e no teste.

In [ ]:
# criar coluna binária de feriados

train['holiday'] = # TODO
test['holiday'] = # TODO

# 5. Regressão

Essa sessão é livre! Tente usar uma regressão linear e um RandomForestRegressor. Será necessário usar algum encoding para as variáveis categóricas. Uma poissível alternativa é utilizar uma feature recente do scikit-learn: [ColumTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html). O ColumnTransformer possibilida a utilização de preprocessadores que atuam em colunas diferentes dentro do mesmo objeto!

Por exemplo, podemos combinar em um só pré-processador um MinMaxScaler que atua nas features numéricas e um OneHotEncoder que atua nas features categóricas:

```python
numerical_features = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
                      'dropoff_latitude', 'Temp.', 'Precip', 'snow', 'Visibility',
                      'delta_lat', 'delta_long', 'haversine_dist',
                      'pickup_speed_nta_hour', 'dropoff_speed_nta_hour']
```

```python
categorical_features = ['store_and_fwd_flag', 'pickup_ntacode', 'dropoff_ntacode',
                        'pickup_neighborhood', 'dropoff_neighborhood', 'vendor_id']
```

```python
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', MinMaxScaler(), numerical_features),
        ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])
```

Agora é só usar o fit_transform:

```python
X = preprocessor.fit_transform(train)
```

Esse processor também pode ser colocado em um pipeline junto com o classificador.


Quando for utilizar o .fit do seu modelo, utilize a transformação np.log1p no target, dessa forma:

```python
model.fit(X, np.log1p(y))
```

Para calcular o RMSLE:

```python
RMSLE = np.sqrt(mean_squared_log_error(y_pred, np.exp(y_pred) - 1))
```

In [ ]:
# BIG TODO
# fitar regressão logística/random forest

# Obrigado!

![](imgs/taxi_ending.jpg)